In [ ]:
# Note: restart runtime after this import before running the augmentations
!pip install -U augly[image]
!sudo apt-get install python3-magic

In [ ]:
import os
import augly.image as imaugs
import augly.utils as utils
from IPython.display import display

# Get input image, scale it down to avoid taking up the whole screen
input_img_path = os.path.join(
    utils.TEST_URI, "image", "inputs", "dfdc_1.jpg"
)

# We can use the AugLy scale augmentation
input_img = imaugs.scale(input_img_path, factor=0.2)
display(input_img)

In [ ]:
# Now we can apply various augmentations to the scaled image!
display(
    imaugs.meme_format(
        input_img,
        caption_height=75,
        meme_bg_color=(0, 0, 0),
        text_color=(255, 255, 255),
    )
)

In [ ]:
"""
You can optionally pass in a metadata list, to which metadata about the
augmentation will be appended, including kwargs, input & output
dimensions, and intensity (defined based on the kwargs for each
augmentation).
"""
meta = []
display(imaugs.shuffle_pixels(input_img, factor=0.3, metadata=meta))
meta

In [ ]:
"""
You can also pass in bounding boxes, which will be transformed along with
the image & included in the metadata (note: you must provide metadata to
get the transformed bboxes)
"""
meta = []
display(
    imaugs.rotate(
        input_img,
        degrees=15,
        metadata=meta,
        bboxes=[(20, 6, 250, 180)],
        bbox_format="pascal_voc",
    )
)
meta

In [ ]:
# For all the augmentations, we have class-based definitions as well as
# functional
meta = []
aug = imaugs.PerspectiveTransform(sigma=20.0)
display(aug(input_img, metadata=meta))
meta

In [ ]:
"""
For some augmentations, we also provide versions that will randomly sample
from a set of parameters (e.g. for ChangeAspectRatio, RandomAspectRatio
samples an emoji from Twitter's Twemoji set which we provide in the augly
package). The metadata will contain the actual sampled param values.
"""
meta = []
aug = imaugs.RandomAspectRatio()
display(aug(input_img, metadata=meta))
meta

In [ ]:
# You can also compose several transformations together
meta = []
aug = imaugs.Compose(
    [
        imaugs.Saturation(factor=2.0),
        imaugs.OverlayOntoScreenshot(
            template_filepath=os.path.join(
                utils.SCREENSHOT_TEMPLATES_DIR, "mobile.png"
            ),
        ),
        imaugs.Scale(factor=0.6),
    ]
)
display(aug(input_img, metadata=meta))
meta

In [ ]:
# AugLy also integrates seamlessly with PyTorch transforms
# Note: you must have torchvision installed, which it is by default in colab
import torchvision.transforms as transforms

aug = transforms.Compose(
    [
        imaugs.Brightness(factor=2.0),
        imaugs.RandomRotation(),
        transforms.ToTensor(),
    ]
)
type(aug(input_img))

In [ ]:
# We also provide a numpy wrapper in case your data is in np.ndarray format
import numpy as np
from augly.image import aug_np_wrapper, overlay_emoji

np_image = np.zeros((300, 300))
# pass in function arguments as kwargs
np_aug_img = aug_np_wrapper(np_image, overlay_emoji, **{'opacity': 0.5, 'y_pos': 0.45})
type(np_aug_img)